- [문제 링크](https://www.acmicpc.net/problem/17472)
    - 삼성 기출문제라고 하네요~
- 점이 면적으로 주어져 있고
    - 모든 면적을 연결하는 다리의 최소 길이를 구하는 문제인데
    - 다리는 중간에 꺾이면 안됨(x방향으로 간다면 x방향 끝에 다른 점이 있어야 함)
        - 즉 다리를 연결한 끝에서 꺾이는 것도 불가능하다는 뜻
    - 다리의 길이는 2 이상이어야 함
    - 위치가 겹치는 경우, 중복된 칸으로 계산하지 않고 별개의 칸으로 계산함

In [5]:
import sys
from collections import deque
sys.setrecursionlimit(int(1e6))
# sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline
N, M = map(int, input().split()) # 세로, 가로의 크기

graph = []
for _ in range(N):
    graph.append(list(map(int, input().split())))


def bfs(row, col):
    
    global area_idx 
    
    dq = deque([])
    loc_lst = [] # 2. 에서 추가
    
    dq.append((row, col))
    land_arr.append((row, col, area_idx))
    graph[row][col] = area_idx
    actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    # 그래프의 영역을 바꾼다
    while dq:
        r, c = dq.popleft()
        for action in actions:
            dr = r + action[0]
            dc = c + action[1]
            
            if dr < 0 or dr >= N:
                continue
            if dc < 0 or dc >= M:
                continue
                
            if graph[dr][dc] == 1:
                dq.append((dr, dc))
                land_arr.append((dr, dc, area_idx)) # 수정 : 모든 영역의 땅 좌표와 영역 번호를 갖는 배열을 이용함 
                graph[dr][dc] = area_idx



# 1. 영역마다 인덱스 값을 다르게 준다.
area_idx = 2
land_arr = [] # 모든 땅의 좌표와 영역 번호 배열
for i in range(N):
    for j in range(M):
        if graph[i][j] == 1:
            bfs(i, j)
            area_idx += 1         
            
# 2. 땅 배열을 이용해서 다른 점까지의 거리를 하나하나 카운트한 뒤, Edge에 더함
moves = [(-1, 0), (1, 0), (0, 1), (0, -1)]
edges = []
for row, col, area in land_arr:
    for move in moves:
        drow, dcol = row + move[0], col + move[1]
        dist = 0
        
        # 다른 좌표로 이동했을 때의 경우의 수는
            # 1. 같은 번호의 땅
            # 2. 바다
            # 3. 다른 번호의 땅
            # 이 되며, 중간에 뭐가 있을지 모르기 때문에 하나하나 세줘야 함
            # 한 방향으로 거리 2 이상인 1이 있다고 해서 두 점 사이의 거리를 이용한다고 생각해보자
            # 근데 그 중간에 뭐가 있을지 모른다는 거임! ((1 0 1 0 1)은 위 조건을 만족하지만, 다리를 연결할 수는 없음)
            # 또한, 1번 움직였을 때는 그 방향으로 쭉 갈 수 있기 때문에 while을 이용할 수 있다.
            
        while True:
            
            if 0 <= drow < N and 0 <= dcol < M:
                
                # 1. 같은 번호의 땅
                if graph[drow][dcol] == area:
                    break

                # 2. 바다
                if graph[drow][dcol] == 0:
                    drow += move[0]
                    dcol += move[1]
                    dist += 1
                    continue

                # 위의 두 조건에서 break과 continue가 들어갔기 때문에,
                # 여기부터는 바다를 건너서 다른 땅에 도착했다라는 조건이 됨.
                # 즉, dist 갱신이 끝난 시점이며, 여기서 dist 조건을 검토할 수 있음
                if dist < 2:
                    break

                another_area = graph[drow][dcol]
                edges.append((dist, area, another_area))
                break

            else: # 이동한 좌표가 그래프를 벗어날 수 있음. 이 경우는 dist를 구할 필요가 없겠죠? 땅이 없으니까
                break

area_counts = area_idx - 2 # 시작 인덱스가 2, 현재 저장된 인덱스는 마지막 영역 번호 + 1 이므로

# 3. union find를 이용한다 - Kruskal이 훨씬 편해보임
parent = [i for i in range(area_counts + 2)]

def find(x):
    if parent[x] != x:
        parent[x] = find(parent[x])
    return parent[x]

def union(a, b):
    if a == b:
        return
    
    a = find(a)
    b = find(b)
    
    if a > b:
        a, b = b, a
        
    parent[b] = a
edges.sort()

ans = 0

for w, a, b in edges:

    if find(a) == find(b):
        continue
        
    union(a, b)
    ans += w

flag = True
for i in range(2, len(parent) - 1): # parent는 2번부터 체크하면 됨 : 땅 번호가 2번부터 시작이었으니까
    if find(parent[i]) != find(parent[i+1]):
        flag = False
        print(-1)
        break
if flag:
    print(ans)

9


# 풀이 후 정리
## 문제
- 각 영역을 연결하는 최소 거리를 구하는 문제임
- 그래프를 최소 거리로 연결한다는 점에서는 MST 문제인데, 각 점을 영역으로 준 것.
---
## 풀이
1. MST 문제니까
    1. 각 영역을 연결하는 간선을 구해야 함
    2. 그래프가 0과 1로만 정의되었기 때문에, 각 영역을 구분할 필요가 있음
2. 따라서, **우선 각 영역을 별도로 구분하기 위해 BFS나 DFS를 이용함.**
    - 이 과정에서 각 영역(땅)의 좌표를 저장하는 배열`loc_arr`을 하나 만들고, `(row, col, area)`로 저장함.
    - 또한, 그래프에 있는 값 `1`을 영역에 해당하는 숫자`2부터`로 바꿔주기도 했음.
3. **Edge 리스트 구하기** :
    - `loc_arr`의 행, 열 좌표에 대해 상하좌우로 1칸씩 움직일 때 경우의 수는 3가지(`같은 번호의 땅`, `바다`,` 다른 번호의 땅`)임
        1. 같은 번호의 땅
            - 연결된 땅이므로 다리를 짓는 조건이 아님. `break`
        2. 바다
            - 다리는 중간에 꺾일 수 없기 때문에, 지금 좌표로 온 이동 방법으로 계속 이동함
            - 그래프를 벗어나면 섬-섬 연결이 아니니까 다리를 짓는 케이스가 아님
            - 다리를 한 칸 지을 수 있으니까 `간선값 + 1` 
            - `continue`
        - 경우 1, 2에 해당하면 이 밑으로 못 옴 : 즉, 여기부터는 다른 번호의 땅에 도달한게 됨
        3. `간선값 < 2`
            - 문제에서 이 경우는 제외하라고 했으니 `break`
        4. (별도의 조건 X : 거리 2 이상이면서 다른 땅에 도달한 경우)
            - (`간선값, 출발한 영역, 도착한 영역`)을 `Edge 리스트`에 추가함
   - 실제 구현은 `for move in moves` 속에서 `while True`를 돌린 모양이 됨
4. 3까지 구현되었다면, 이 다음부터는 `Kruskal Algorithm`과 완전히 동일함
    - 간선들을 오름차순으로 정렬하고, `parent` 배열을 만들고, `union, find`함수를 만들고..
        - 나는 영역을 `2`부터 구현했기 때문에 `parent` 배열을 만들 때 `영역 갯수 + 2`의 크기로 만들었음
    - 마지막에 모든 `parent[2:]`는 같은 값을 가리키는가?를 살펴볼 때,
    - `find(parent[idx]) == find(parent[idx + 1])`로 조건을 넣어줘야 하는 것만 유의하자
        - `union`과정에서 기존에 연결된 노드들이 가리키는 값은 변하지 않음(`find`로 다시 한 번 봐줘야 함)

# 푸는 과정에서 생각한 것들

## 뭐가 필요할까?(설계)
1. 점들을 연결하는 가장 짧은 길이를 구하는 문제니까 최소 신장 트리일 거임
2. 그러면 두 영역의 가장 짧은 거리를 구하는 알고리즘이 있어야 함
3. 두 영역이 주어졌다면
    1. 우선 모든 영역에 대해 dfs나 bfs를 돌려서 visited 값을 바꿔줘야 할 듯?
        - 두 영역을 연결했다면 같은 숫자를 가리키도록 구성원을 바꿔주면 됨
    2. 연결할 수 있는지 여부
        - 영역을 구성하는 좌표 중 `row`가 같거나 `col`이 같아야 함
        - `row`, `col` 중 하나가 같더라도, 그 두 점 사이의 거리가 1이면 X
    3. ~연결하는 점들이 여러 개일 수 있음~
        - ~이 경우 연결 조건을 성립하는 점들의 쌍에 대해 거리(인덱스 차이)를 구하고, 최솟값을 갱신하면 됨~
        - ~모든 점에 대해 성립한다면 종료하고, 다리의 길이를 결과값에 더함~

## 풀다 보니
#### ~1. A.는 필요 없을 것 같음~
- ~dfs나 bfs가 필요하긴 한데, 각 영역을 구성하는 좌표 리스트만 모아주면 될 것 같다.~
- 각 영역을 구분할 필요가 있기 때문에 A.는 필요한 과정이며, 여기서 (땅 좌표, 영역 번호)를 구성하는 리스트도 만든다.
#### 2. 각 영역을 잇는 간선을 구현할 때 필요한 것
- 원래는 "모든 점"에 대한 간선을 구했음
```
ex ) (2, 0) (1, 0)이 있고 다른 영역에 (4, 0)이 있다고 하자
그러면 모든 점에 대해 거리를 구하면 각각 2와 3인데
"땅을 지나는 다리가 없기 때문에" (1, 0)은 해당되지 않아야 맞음
```
- 따라서 어떤 좌표가 주어졌을 때, 해당 인덱스 방향으로 +1 했을 때 같은 영역을 구성하는 값이 있는지 확인해야 함
- 그냥 값이 있냐/없냐 여부로 따져도 될 듯?

## 틀렸거나 시간초과됐거나 점검

### 1. 가장 마지막에 제대로 연결됐는지에 대한 조건을 다시 수정함
```python
# if ans == 0:
#     print(-1)
# else:
#     print(ans)

flag = True
for i in range(len(parent) - 1):
    if parent[i] != parent[i + 1]:
        flag = False
        print(-1)
        break
if flag:
    print(ans)
```
- 수정 결과 ) 시간초과(Python 3), 메모리 초과(Pypy 3)

### 2. 역시 4중 for문이 문제일 듯?
- 간선들을 얻는 방법을 다시 생각해보자
- 기존 방법
```
1. 영역별로 땅의 좌표를 리스트에 따로 담는다.
2. 각 영역의 점들에 대해 반복문을 돌린다 : 두 점이 연결되는 방향으로 바로 다음 칸의 영역이 없을 때 연결됨  

이 방법의 문제는 1 0 1 0 1 같은 상황에 대처를 못하는 것과, 4중 for문이 사용된다는 것임(영역 2개, 영역 내의 각 점들에 대해)  
```

### 개선) (풀이 참고)  
- 두 점이 있더라도, 그 사이에 점이 어떻게 올 지 모르기 때문에 다리가 되는 점들(0) 하나하나를 다 따져봐야 한다.
```
1. BFS를 통해 땅의 좌표를 담을 때, (row, col, AreaNo) 순으로 담는다.
2. 모든 땅의 좌표에 대해 반복문을 돌림
    - moves를 구현하고
    - dx, dy로 이동한 뒤
        0. dx, dy는 0과 m(n) 사이에 있어야 함
        1. 같은 번호의 땅이라면 더 반복할 필요 X
        2. 바다라면 dx, dy값에 각각의 move 값을 더하고 거리 값도 더함
            - 다음값에 대해 다시 반복
        3. 결과적으로 dist 값이 2보다 작다면 이건 해당 X
        4. 다른 번호의 땅을 만나게 된다면 edge 행렬에 (dist, 시작땅, 도착땅)을 append함
```
- 이 과정은 
    1. 땅 좌표 배열 반복
    2. move 배열 반복
    3. 다른 땅에 닿을 때까지 반복
    - 으로, 3중 for문이 구성되며 move는 4개니까 실제로는 더 적을 거임

#### 3. 마무리
- 마지막에 `parent` 행렬을 점검할 때는 `find` 함수를 넣어서 검색해야 한다
- 알다시피 `union`은 기존에 연결되어 있던 노드들의 `parent`값을 갱신하지 않기 때문이다.